# 🚀 ComfyUI + Wan 2.2（阿里通义万相 2.2）一键安装与启动（Google Colab）

> 最近更新：2025-08-21  
> 说明：本 Notebook 按照 Reddit 帖子整理的模型清单与官方示例制作，包含 **Wan 2.2 14B/5B** 所需模型下载与启动脚本，并附带 **Cloudflared 公网地址** 打开 ComfyUI。  
> *建议使用 Colab（GPU 模式）运行；首次运行需要下载较大模型。*

**参考来源**（供你核对模型/工作流放置路径）：  
- Reddit 帖子：**Wan2.2 Workflows, Demos, Guide, and Tips!**（列出模型文件与放置目录）  
- Comfy 官方示例与下载清单：**ComfyUI_examples / Wan 2.2 Models**  
- Comfy 官方文档：**Wan 2.2 视频生成（原生工作流示例）**

> 小贴士：14B 模型显存占用较高；显存较小时推荐先用 **5B（TI2V）** 或在 ComfyUI 中开启 offload 相关选项的工作流。

In [ ]:
#@title 第 0 步：环境/GPU 检查（可选）
# 使用前请先在 Colab 菜单：运行时 -> 更改运行时类型 -> 硬件加速器 选择 “GPU”
# 该单元仅做 GPU & 环境信息检查，便于排错。

import os, sys, platform, shutil, subprocess

def run(cmd):
    print(f"\n$ {cmd}")
    p = subprocess.run(cmd, shell=True, check=False, text=True, capture_output=True)
    print(p.stdout or "")
    if p.returncode != 0:
        print(p.stderr or "")
    return p.returncode

print("Python:", sys.version)
print("Platform:", platform.platform())

run("nvidia-smi || echo '未检测到 GPU（如需生成视频请在 Colab 选择 GPU）'")
run("python -m pip -V")

In [ ]:
#@title 第 1 步：安装依赖 & 克隆 ComfyUI（必做）
%%bash
# 安装基础依赖并拉取 ComfyUI 与 ComfyUI-Manager
set -e

echo "更新 apt 包索引，并安装 ffmpeg、git、aria2（多线程下载器）..."
apt-get update -y -qq
apt-get install -y -qq ffmpeg git aria2

cd /content

if [ ! -d "ComfyUI" ]; then
  echo "克隆 ComfyUI 仓库..."
  git clone https://github.com/comfyanonymous/ComfyUI.git
else
  echo "已存在 ComfyUI，执行更新..."
  cd ComfyUI && git pull --ff-only && cd ..
fi

# 安装 ComfyUI-Manager（方便后续安装/管理自定义节点）
if [ ! -d "/content/ComfyUI/custom_nodes/ComfyUI-Manager" ]; then
  echo "安装 ComfyUI-Manager 插件..."
  git clone https://github.com/ltdrdata/ComfyUI-Manager.git /content/ComfyUI/custom_nodes/ComfyUI-Manager
fi

echo "安装 Python 依赖（包括 PyTorch CUDA 12.1 版）..."
pip -q install --upgrade pip wheel setuptools
pip -q install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
pip -q install -r /content/ComfyUI/requirements.txt
echo "✅ 基础环境准备完成"

In [ ]:
#@title 第 2 步（可选）：模型目录持久化到 Google Drive
%%bash
# 可选：把模型持久化到 Google Drive，避免下次重复下载
# 运行本单元后，会在 /content/ComfyUI/models 处建立指向你云盘的软链接
set -e

read -p "是否要把模型保存到 Google Drive? 输入 y 继续（否则按回车跳过）: " ans
if [[ "$ans" == "y" || "$ans" == "Y" ]]; then
  python - <<'PY'
from google.colab import drive
drive.mount('/content/drive')
PY
  mkdir -p /content/drive/MyDrive/ComfyUI/models
  rm -rf /content/ComfyUI/models
  ln -s /content/drive/MyDrive/ComfyUI/models /content/ComfyUI/models
  echo "✅ 已将模型目录链接到 Google Drive: /content/drive/MyDrive/ComfyUI/models"
else
  echo "跳过持久化，模型将保存在当前会话：/content/ComfyUI/models"
fi

In [ ]:
#@title 第 3 步：自动下载 Wan 2.2 模型到正确目录（必做）
%%bash
# 下载 Wan 2.2 必需模型（来自 Hugging Face Comfy-Org/Wan_2.2_ComfyUI_Repackaged）
# Reddit 帖子中给出了清单与放置目录，本单元已为你自动放置到正确路径。
set -e

BASE=/content/ComfyUI/models
DM=$BASE/diffusion_models
TE=$BASE/text_encoders
VAE=$BASE/vae
LORA=$BASE/loras

mkdir -p "$DM" "$TE" "$VAE" "$LORA"

# ---- Diffusion 模型 ----
# 14B T2V / I2V（高噪 + 低噪）
aria2c -c -x16 -s16 -k1M -d "$DM" -o wan2.2_t2v_high_noise_14B_fp8_scaled.safetensors \
  https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/diffusion_models/wan2.2_t2v_high_noise_14B_fp8_scaled.safetensors

aria2c -c -x16 -s16 -k1M -d "$DM" -o wan2.2_t2v_low_noise_14B_fp8_scaled.safetensors \
  https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/diffusion_models/wan2.2_t2v_low_noise_14B_fp8_scaled.safetensors

aria2c -c -x16 -s16 -k1M -d "$DM" -o wan2.2_i2v_high_noise_14B_fp8_scaled.safetensors \
  https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/diffusion_models/wan2.2_i2v_high_noise_14B_fp8_scaled.safetensors

aria2c -c -x16 -s16 -k1M -d "$DM" -o wan2.2_i2v_low_noise_14B_fp8_scaled.safetensors \
  https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/diffusion_models/wan2.2_i2v_low_noise_14B_fp8_scaled.safetensors

# 5B TI2V（一个文件即可同时支持 T2V & I2V）
aria2c -c -x16 -s16 -k1M -d "$DM" -o wan2.2_ti2v_5B_fp16.safetensors \
  https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/diffusion_models/wan2.2_ti2v_5B_fp16.safetensors

# ---- Text Encoder ----
aria2c -c -x16 -s16 -k1M -d "$TE" -o umt5_xxl_fp8_e4m3fn_scaled.safetensors \
  https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/text_encoders/umt5_xxl_fp8_e4m3fn_scaled.safetensors

# ---- VAEs ----
# 14B 使用 Wan 2.1 的 VAE
aria2c -c -x16 -s16 -k1M -d "$VAE" -o wan_2.1_vae.safetensors \
  https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/vae/wan_2.1_vae.safetensors

# 5B 使用 Wan 2.2 的 VAE
aria2c -c -x16 -s16 -k1M -d "$VAE" -o wan2.2_vae.safetensors \
  https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/vae/wan2.2_vae.safetensors

# ---- 可选：LightX2V LoRA（更快/更稳的 4~8 步采样方案） ----
# 放入 /ComfyUI/models/loras
aria2c -c -x16 -s16 -k1M -d "$LORA" -o Wan21_T2V_14B_lightx2v_cfg_step_distill_lora_rank32.safetensors \
  https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Wan21_T2V_14B_lightx2v_cfg_step_distill_lora_rank32.safetensors

aria2c -c -x16 -s16 -k1M -d "$LORA" -o lightx2v_I2V_14B_480p_cfg_step_distill_rank128_bf16.safetensors \
  https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Lightx2v/lightx2v_I2V_14B_480p_cfg_step_distill_rank128_bf16.safetensors

echo "✅ 模型下载完成，已放置到 /content/ComfyUI/models 下的对应目录。"

In [ ]:
#@title 第 4 步（可选）：下载官方 Wan 2.2 工作流 JSON
%%bash
# 可选：下载官方示例工作流（JSON），便于一键加载
set -e

WF_DIR=/content/ComfyUI/user/default/workflows/Wan2.2_examples
mkdir -p "$WF_DIR"

# 5B：文本生视频 / 图生视频
aria2c -c -x16 -s16 -k1M -d "$WF_DIR" -o text_to_video_wan22_5B.json \
  https://comfyanonymous.github.io/ComfyUI_examples/wan22/text_to_video_wan22_5B.json

aria2c -c -x16 -s16 -k1M -d "$WF_DIR" -o image_to_video_wan22_5B.json \
  https://comfyanonymous.github.io/ComfyUI_examples/wan22/image_to_video_wan22_5B.json

# 14B：文本生视频 / 图生视频
aria2c -c -x16 -s16 -k1M -d "$WF_DIR" -o text_to_video_wan22_14B.json \
  https://comfyanonymous.github.io/ComfyUI_examples/wan22/text_to_video_wan22_14B.json

aria2c -c -x16 -s16 -k1M -d "$WF_DIR" -o image_to_video_wan22_14B.json \
  https://comfyanonymous.github.io/ComfyUI_examples/wan22/image_to_video_wan22_14B.json

echo "✅ 工作流已保存到：$WF_DIR
在 ComfyUI 界面左上角 Load -> Workflows 里即可看到。"

In [ ]:
#@title 第 5 步：启动 ComfyUI（后台）
%%bash
# 启动 ComfyUI（后台运行），日志写入 /content/comfy_log.txt
set -e
cd /content/ComfyUI
nohup python main.py --listen 0.0.0.0 --port 8188 > /content/comfy_log.txt 2>&1 &
sleep 2
echo "✅ ComfyUI 已在后台启动，端口 8188"
echo "日志位置：/content/comfy_log.txt"

In [ ]:
#@title 第 6 步：生成 ComfyUI 公网访问链接（Cloudflared）
%%bash
# 通过 Cloudflared 暴露公网地址（运行后请在输出里寻找 https://xxxxx.trycloudflare.com）
set -e
cd /content
if [ ! -f "/content/cloudflared" ]; then
  echo "下载 cloudflared 二进制..."
  wget -q -O /content/cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
  chmod +x /content/cloudflared
fi

echo "启动 Cloudflared 隧道...（按停止按钮可终止）"
/content/cloudflared tunnel --url http://127.0.0.1:8188 --no-autoupdate

## 常见问题（FAQ）

- **Q：显存不够怎么办？**  
  A：优先尝试 **5B（TI2V）** 工作流，或在官方工作流中开启 offload（例如将 VAE/部分层放到 CPU）。

- **Q：工作流提示缺少节点？**  
  A：请先在 **ComfyUI-Manager** 里检查更新；Wan 2.2 原生已集成，无需额外的自定义节点。若你使用社区工作流（如 LightX2V 特化），请根据其说明补装对应节点。

- **Q：Cloudflared 长时间无链接或打不开？**  
  A：重启该单元；或尝试切换到“运行时 -> 断开并删除运行时”，再从上到下重跑。也可改用其它隧道方案（如 localtunnel）。

- **Q：如何导入本 Notebook 下载的示例工作流？**  
  A：ComfyUI 左上角 **Load -> Workflows**，选择 `Wan2.2_examples` 下的 JSON。